In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import h5py
import os
import shutil

#sys.path.append("../../W_KU/Fullerene/Thesis_Folding_Carbon/data/C60_closing/")
sys.path.append("unfolding/")
sys.path.append("src/electronic_structure/GAUSSIAN/")
#sys.path.append("../thesis-carbon-folding/functions")
#sys.path.append("../thesis-carbon-folding/Unfolding/Application_functions")
#sys.path.append("../results/hdf5/")
#sys.path.append("../fullerene-unfolding/")

from Application_functions import read_unfolding, fit_all_planes, read_geometry
from geometry_functions import detect_minimal_distance
from Unfolding import Unfolding
from functions_folding import *
from interpolation import interpolated_values

dual_unfolding, graph_unfolding, graph_unfolding_faces, vertices_final, bonds_toBe, lengths_toBe, angles_f, opt_geom, halogen_positions, neighbours, graph_faces = read_unfolding("data/unfolding_graph/C60data.h5")

f = open("data/header/header_cam-b3lyp_6-31star")
header_b3lyp = f.read()
f.close()


In [3]:
C60 = Unfolding(dual_unfolding, graph_unfolding_faces, graph_faces, graph_unfolding, neighbours, halogen_positions=halogen_positions, root_node=0, bonds_toBe=bonds_toBe, angles_f=angles_f)
#C60.vertex_coords = geometry
dual_unfolding

root_node = 0, root_face = [0, 1, 2, 3, 4, 5]
The intersection of the two faces are: {0, 1}
The intersection of the two faces are: {2, 3}
The intersection of the two faces are: {4, 5}
The intersection of the two faces are: {6, 7}
The intersection of the two faces are: {8, 7}
The intersection of the two faces are: {9, 10}
The intersection of the two faces are: {10, 11}
The intersection of the two faces are: {12, 13}
The intersection of the two faces are: {13, 14}
The intersection of the two faces are: {16, 17}
The intersection of the two faces are: {24, 23}
The intersection of the two faces are: {30, 31}
The intersection of the two faces are: {38, 39}
The intersection of the two faces are: {42, 43}
The intersection of the two faces are: {46, 47}
dual hinges:  [[0, 1], [0, 2], [0, 3], [1, 4], [1, 5], [2, 6], [2, 7], [3, 8], [3, 9], [4, 10], [6, 11], [8, 12], [10, 13], [11, 14], [12, 15]]
cubic hinges: [[0, 1], [2, 3], [4, 5], [6, 7], [7, 8], [9, 10], [10, 11], [12, 13], [13, 14], [16, 17

[[1, 2, 3],
 [0, 4, 5],
 [0, 6, 7],
 [0, 8, 9],
 [10, 5, 1],
 [4, 1],
 [11, 7, 2],
 [2, 6],
 [9, 3, 12],
 [3, 8],
 [13, 4],
 [14, 6],
 [15, 8],
 [10],
 [11],
 [12]]

In [4]:
def calculate_final_angles(vertices_final,graph_unfolding_faces, hinges):
    final_midpoints = np.zeros([len(graph_unfolding_faces),3])
    for faceId, face in enumerate(graph_unfolding_faces):
        final_midpoints[faceId] = np.sum(vertices_final[face],axis=-2)

    hinge_midpoints = np.sum(vertices_final[[tuple(hinges[1])]],axis=-2) / 2.
    hinge_legs = final_midpoints[[tuple(hinges[0])]] - np.repeat(hinge_midpoints[:,np.newaxis,:],2,axis=-2)
    angles = np.pi - angle_vec(hinge_legs[:,0,:],hinge_legs[:,1,:],degrees=False)
    return angles

In [5]:
def graph_from_final_geometry(vertices_final):
    graph = np.zeros_like(vertices_final,dtype=int)
    distances = np.zeros([len(vertices_final),len(vertices_final)])
    distances[0,0] = 1e3
    for i in range(len(vertices_final)):
        for j in range(i):
            distances[i,j] = np.sqrt(np.sum((vertices_final[i] - vertices_final[j])**2))
            distances[j,i] = distances[i,j]
            distances[i,i] = 1e3
    for i in range(len(vertices_final)):
        graph[i] = np.argsort(distances[i])[:3]
    return graph

In [6]:
from geometry_functions import mean_normal
X = vertices_final
closed_normals = np.zeros((len(graph_unfolding_faces),3),dtype=float)
closed_normals[C60.face_type[0]] = mean_normal(X, C60.pentagons)
closed_normals[C60.face_type[1]] = mean_normal(X, C60.hexagons)
angles_final = [angle_vec(closed_normals[u],closed_normals[v],degrees=False) for u,v in C60.hinges[0]]
np.pi - np.array(angles_final)

array([2.82324916, 2.60066408, 2.72332814, 2.5831659 , 2.26439429,
       0.85688439, 2.41168656, 1.81574428, 2.07556686, 1.21240265,
       1.66691542, 1.5519155 , 2.17150802, 2.40932837, 0.91323892])